<a href="https://colab.research.google.com/github/shuuuuyu/114-1-Programing-Language/blob/main/hw_1_%E6%97%A5%E5%B8%B8%E6%94%AF%E5%87%BA%E9%80%9F%E7%AE%97%E8%88%87%E5%88%86%E6%94%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 記帳軟體作業說明

## 目標
設計一個簡易的記帳系統，功能包含：
1. **AA 功能**：計算每人分攤金額，並記錄至 Google Sheet。
2. **記帳功能**：記錄消費的時間、種類、金額、項目名稱與備註。
3. **AI 點子板塊**：分析最近一週的消費，提供統計與建議。

---

## 功能板塊

### 1. AA 功能
- 輸入總金額。
- 選擇服務費百分比（0%、5%、10%）。
- 輸入分攤人數。
- 額外輸入餐點或描述文字。
- 系統會計算每人應付金額，並寫入 Google Sheet 的「外食」欄位。

### 2. 記帳功能
- 輸入以下欄位：
  - 消費種類：外食、買菜、日用品
  - 金額
  - 項目名稱（如餐廳名稱、材料名稱、日用品名稱）
  - 備註
- 記錄會直接寫入 Google Sheet。
- Google Sheet 設定為「知道連結的使用者可以編輯」，無需服務帳號憑證。

### 3. AI 點子板塊
- 顯示最近一週的消費：
  - 每個種類的總金額
  - 每筆消費明細
- AI 分析功能：
  - 後續可使用模型提供消費習慣分析與建議（例如「外食過多」）

---

## 使用方式
1. 開啟程式，會跳出 Google 登入授權視窗，登入後可操作 Google Sheet。
2. 在 **AA 功能** 或 **記帳功能** 填入資料，按下送出按鈕，即可寫入 Google Sheet。
3. 前往 **AI 點子板塊**，點擊「查看最近一週消費」以統計消費資料。
4. AI 建議按鈕目前保留，後續可補上分析模型。

---

## Google Sheet 格式
- Sheet 欄位：
  - 時間
  - 種類
  - 金額
  - 項目名稱
  - 備註
- 系統會自動將資料新增至該 Sheet。

---

## 備註
- 程式使用 Google OAuth 登入，不需 Service Account JSON。
- AA 功能會將分攤後金額寫入 Google Sheet，方便統計。
- 程式使用 Gradio 建立簡易網頁介面，方便操作。


## AI 測試

In [1]:
import os
from google.colab import userdata
from openai import OpenAI

In [9]:
!pip install openai
!pip install gradio

In [18]:
title = "理財消費分析器"

system = '''你是一個消費與理財分析專家，專門分析使用者的消費數據。請以專業、清晰且友善的語氣，對以下消費數據提供建議。

分析內容應包括：
1. 消費習慣觀察（例如：外食頻率過高、日用品花費分布）
2. 主要花費項目與金額占比
3. 節省或調整建議
4. 理財提醒與下一步行動建議

請用台灣熟悉的繁體中文回答。
請保持語氣親切、實用，不要加不必要的示例或說明文字。
'''

description = '''
想要了解自己的消費習慣並獲得理財建議？
這個生成器專為「消費分析與理財建議」設計。
只要提供最近一週的消費數據，它就會生成消費統計、問題觀察與改善建議，幫助你更好地掌控花費。
'''


In [21]:
messages = [{"role":"system",
             "content":system}]

In [27]:
api_key = userdata.get('groq-pl')
model = "qwen/qwen3-32b"
base_url="https://api.groq.com/openai/v1"

In [15]:
client = OpenAI(
    base_url = base_url # 如用 OpenAI 不需要這一行
)

In [17]:
os.environ['OPENAI_API_KEY']=api_key

In [19]:
def mychatbot(prompt):
    messages.append({"role": "user", "content": prompt})
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        )
    reply = chat_completion.choices[0].message.content
    return reply

In [28]:
iface = gr.Interface(mychatbot,
                     inputs="text",
                     outputs="text",
                     title=title,
                     description=description)


iface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://db4b0cba9a93ac52d0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Using existing dataset file at: .gradio/flagged/dataset1.csv
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://db4b0cba9a93ac52d0.gradio.live


## 主要程式

In [37]:
# ==============================
# 掛載與套件載入
# ==============================
import pandas as pd
import gradio as gr
import datetime
import gspread
from google.colab import auth
from google.auth.transport.requests import Request
from google.auth import default

# ==============================
# Google 帳號授權
# ==============================
auth.authenticate_user()   # 會跳出 Google 登入
creds, _ = default()
gc = gspread.authorize(creds)

# 你的 Google Sheet ID
SHEET_ID = "1MbCem2-X-1vTViIxN9j5Eu_C6fh-AtZQkTaIrzOE7Ok"

# 打開第一個工作表
sh = gc.open_by_key(SHEET_ID)
worksheet = sh.sheet1


# ==============================
# 板塊一：AA 功能
# ==============================
def aa_function(total_amount, service_fee_percent, people_count, description):
    if service_fee_percent is None:
        service_fee_percent = 0
    if people_count is None or people_count <= 0:
        people_count = 1

    final_amount = total_amount * (1 + service_fee_percent / 100)
    per_person = final_amount / people_count

    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    new_record = [now, "外食", per_person, description, f"含{service_fee_percent}%服務費, {people_count}人分攤"]

    worksheet.append_row(new_record)

    return f"總金額 {final_amount:.2f} 元，每人分攤 {per_person:.2f} 元，已新增到 Google Sheet。"


# ==============================
# 板塊二：記帳功能
# ==============================
def record_expense(expense_type, amount, item_name, note):
    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    new_record = [now, expense_type, amount, item_name, note]

    worksheet.append_row(new_record)

    return f"成功記錄：{expense_type} - {item_name} - {amount} 元"

'''
# ==============================
# 板塊三：AI 點子
# ==============================
def analyze_week():
    data = worksheet.get_all_records()
    df = pd.DataFrame(data)
    df["時間"] = pd.to_datetime(df["時間"], errors="coerce")

    one_week_ago = datetime.datetime.now() - datetime.timedelta(days=7)
    recent_df = df[df["時間"] >= one_week_ago]

    summary = recent_df.groupby("種類")["金額"].sum().reset_index()
    detail = recent_df[["時間", "種類", "金額", "項目名稱"]]

    return summary, detail


def ai_suggestion():
    return "AI 建議功能尚未實作，這裡會顯示模型輸出的建議。"
    '''


# ==============================
# Gradio 介面
# ==============================
with gr.Blocks() as demo:
    gr.Markdown("## 記帳軟體")

    # 板塊一
    with gr.Tab("AA 功能"):
        total_input = gr.Number(label="總金額")
        fee_input = gr.Dropdown([0, 5, 10], label="服務費 (%)", value=0)
        people_input = gr.Number(label="分攤人數", value=1)
        desc_input = gr.Textbox(label="餐點/描述")
        aa_btn = gr.Button("送出")
        aa_output = gr.Textbox(label="結果")
        aa_btn.click(fn=aa_function, inputs=[total_input, fee_input, people_input, desc_input], outputs=aa_output)

    # 板塊二
    with gr.Tab("記帳功能"):
        type_input = gr.Dropdown(["外食", "買菜", "日用品"], label="種類")
        amount_input = gr.Number(label="金額")
        item_input = gr.Textbox(label="項目名稱")
        note_input = gr.Textbox(label="備註")
        record_btn = gr.Button("記錄消費")
        record_output = gr.Textbox(label="結果")
        record_btn.click(fn=record_expense, inputs=[type_input, amount_input, item_input, note_input], outputs=record_output)

    # 板塊三
    '''
    with gr.Tab("AI 點子"):
        analyze_btn = gr.Button("查看最近一週消費")
        summary_output = gr.Dataframe(label="各種類總結")
        detail_output = gr.Dataframe(label="詳細消費")
        analyze_btn.click(fn=analyze_week, inputs=[], outputs=[summary_output, detail_output])

        suggestion_btn = gr.Button("AI 給建議")
        suggestion_output = gr.Textbox(label="AI 建議")
        suggestion_btn.click(fn=ai_suggestion, outputs=suggestion_output)
    '''
    with gr.Tab("AI 點子"):
        gr.Markdown("### 💡 智能消費分析")

        with gr.Row():
            with gr.Column():
                analyze_btn = gr.Button("📊 查看最近一週消費", variant="primary")
                suggestion_btn = gr.Button("🤖 AI 給建議", variant="secondary")

        with gr.Row():
            with gr.Column():
                summary_output = gr.Dataframe(label="📈 各種類總結")
            with gr.Column():
                detail_output = gr.Dataframe(label="📋 詳細消費")

        suggestion_output = gr.Textbox(
            label="🎯 AI 建議",
            lines=10,
            placeholder="點擊 'AI 給建議' 按鈕獲取個人化理財建議..."
        )

        # 事件綁定
        analyze_btn.click(
            fn=analyze_week,
            inputs=[],
            outputs=[summary_output, detail_output]
        )

        suggestion_btn.click(
            fn=ai_suggestion,
            inputs=[],
            outputs=suggestion_output
        )



# 啟動
demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://83a01317c4bce8838e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [36]:
# ==============================
# 板塊三：AI 點子
# ==============================
def analyze_week():
    data = worksheet.get_all_records()
    df = pd.DataFrame(data)
    df["時間"] = pd.to_datetime(df["時間"], errors="coerce")

    one_week_ago = datetime.datetime.now() - datetime.timedelta(days=7)
    recent_df = df[df["時間"] >= one_week_ago]

    summary = recent_df.groupby("種類")["金額"].sum().reset_index()
    detail = recent_df[["時間", "種類", "金額", "項目名稱"]]

    return summary, detail

def format_data_for_ai(summary_df, detail_df):
    """將 DataFrame 轉換成 AI 可讀的文字格式"""
    # 處理總結數據
    summary_text = "【最近一週消費總結】\n"
    total_amount = 0
    for _, row in summary_df.iterrows():
        amount = row["金額"]
        category = row["種類"]
        summary_text += f"- {category}: ${amount:,.0f}\n"
        total_amount += amount
    summary_text += f"\n總計: ${total_amount:,.0f}\n\n"

    # 處理詳細數據
    detail_text = "【詳細消費記錄】\n"
    for _, row in detail_df.iterrows():
        time = row["時間"].strftime("%m/%d") if pd.notna(row["時間"]) else "未知日期"
        category = row["種類"]
        amount = row["金額"]
        item = row["項目名稱"]
        detail_text += f"{time} - {category} - ${amount:,.0f} - {item}\n"

    return summary_text + detail_text

def ai_suggestion():
    """獲取 AI 建議"""
    try:
        # 先獲取最近一週的數據
        summary, detail = analyze_week()

        # 檢查是否有數據
        if summary.empty:
            return "沒有找到最近一週的消費記錄，請先添加一些消費數據。"

        # 將數據格式化為文字
        formatted_data = format_data_for_ai(summary, detail)

        # 準備 AI 分析
        system_prompt = '''你是一個消費與理財分析專家，專門分析使用者的消費數據。請以專業、清晰且友善的語氣，對以下消費數據提供建議。

分析內容應包括：
1. 消費習慣觀察（例如：外食頻率過高、日用品花費分布）
2. 主要花費項目與金額占比
3. 節省或調整建議
4. 理財提醒與下一步行動建議

請用台灣熟悉的繁體中文回答。
請保持語氣親切、實用，不要加不必要的示例或說明文字。
'''

        # 調用 AI API
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"請分析以下消費數據並提供建議：\n\n{formatted_data}"}
        ]

        chat_completion = client.chat.completions.create(
            messages=messages,
            model=model,
        )

        reply = chat_completion.choices[0].message.content
        return reply

    except Exception as e:
        return f"AI 分析過程中發生錯誤: {str(e)}"

